In [1]:
import gsampler as gs
import numpy as np
import pandas as pd
from light.util import selectDates
from IPython.display import display

Deprecated: use ginkgo.Repo instead


###### Using multithread

In [2]:
import multiprocessing
import multiprocessing.pool
NUM_THREADS = 8
class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

##### Set symbol

In [3]:
symbol ='BRN_G0.ICE'
colo = 'ice_quincy_only'
dates = selectDates('20171101~20171130')[:8]
s = '/share/ginkgo/signals/yudis/brn/libBRNbkGrd_BkGr.Xlinear-Filtered-20160816.20160930.v23566.20170917.205108_b23545.so /share/ginkgo/signals/zhaoyan/brn/libBRNSignalBk4v2_price.20170201.20170405.v22187.20170705.115456_b22187.so /share/ginkgo/signals/tian/BRN/libtrd3_eop.filter.target.tm.smt.max.ret.lasso.20170201.20170405.v21801.20170619.121249_b21801.so'
signals = s.split(' ')
TICK_SIZE = 0.01

##### Design features
###### Here shows a template for generating data using multi-thread

In [4]:
def get_raw_features(symbol, colo, date, signals, start_time, end_time):
    features = []
    for i in range(4):
        features.append(gs.FeatureAskPrice(symbol = symbol, level = i))
        features.append(gs.FeatureAskQty(symbol = symbol, level = i))
    for i in range(4):
        features.append(gs.FeatureBidPrice(symbol = symbol, level = i))
        features.append(gs.FeatureBidQty(symbol = symbol, level = i))
    features.append(gs.FeatureTradePrice(symbol = symbol))
    features.append(gs.FeatureTradeSide(symbol = symbol))
    features.append(gs.FeatureTradeSize(symbol = symbol))
    features.append(gs.FeatureTickerIdCast())
    wts = [1.]*len(signals)
    sampler = gs.SamplerRunner(features = features, signals=signals, weights=wts, 
                               dates = [date], colo = colo, cb_version = 'v2')
    sampler.rerun()
    df = sampler.load_data_df()
    df = df.between_time(start_time=start_time, end_time=end_time)
    df = df.fillna(0.)
    df.columns = ['ap0', 'az0', 'ap1', 'az1', 'ap2' ,'az2' ,'ap3' ,'az3' , 
                  'bp0', 'bz0', 'bp1', 'bz1', 'bp2', 'bz2' ,'bp3' ,'bz3' ,
                  'tp', 'ts', 'tv', 'a', 'b', 'time',
                  'sigA','sigB','sigC','sigD','sct']
    df.drop('a', axis=1, inplace=True)
    df.drop('b', axis=1, inplace=True)
    df.drop('sct', axis=1, inplace=True)
    return df

In [5]:
def wrapper_get_raw_features(args):
    return get_raw_features(*args)

In [6]:
args = []
for date in dates:
    args.append((symbol, colo, date, signals, '02:02:00', '15:15:00'))

In [7]:
pool = MyPool(NUM_THREADS)
dfs = pool.map(wrapper_get_raw_features, args)
pool.close()
pool.join()

--- set codebase: ginkgo
--- set codebase: ginkgo
--- set codebase: ginkgo
--- set codebase: ginkgo
--- set codebase: ginkgo
--- set codebase: ginkgo
--- set codebase: ginkgo
--- set codebase: ginkgo
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /tmp/zebo/tmp.signal_symbol.682f3ff78618980d4f48c3ae692342a50c5d4d18.cfg
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /tmp/zebo/tmp.signal_symbol.33e752e8e0ab73dfabcb19ba4c6075bcf8c7447b.cfg
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /tmp/zebo/tmp.signal_symbol.6164a7d2a91bb10c30e2fb9262bebc5cc47c6f57.cfg
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /tmp/zebo/tmp.signal_symbol.e2b719bc99edbba319f5f02772776a9f50299ae4.cfg
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /tmp/zebo/tmp.signal_symbol.d8777b75b90b13f4ee3451775c67977407f6f42d.cfg
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /tmp/zebo/tmp.signal_symbol.24ca1bf0624195dd3d2c3a431adfaa9adc8c42ef.cfg
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /tmp/zebo/tm

BRN_G0.ICE
Debug  (PY: /gexe/signalutils.py:52:getSymbols)  Subscribing symbols:  
CL_G0.CME
BRN_G0.ICE
CL_G0.CME
+++ (0:00:00.634275) /share/ginkgo/bin/B13601/signal_symbols.exe -c /tmp/zebo/tmp.signal_symbol.d8777b75b90b13f4ee3451775c67977407f6f42d.cfg  
+++ (0:00:00.694135) /share/ginkgo/bin/B13601/signal_symbols.exe -c /tmp/zebo/tmp.signal_symbol.24ca1bf0624195dd3d2c3a431adfaa9adc8c42ef.cfg  
*** Sampling 1/1 <ice_quincy_only> 2:00~16:00 CL_G0.CME BRN_G0.ICE x22  
Debug  (PY: /gexe/signalutils.py:52:getSymbols)  Subscribing symbols:  
BRN_G0.ICE
CL_G0.CME
Debug  (PY: /gexe/signalutils.py:52:getSymbols)  Subscribing symbols:  
+++ (0:00:00.575376) /share/ginkgo/bin/B13601/signal_symbols.exe -c /tmp/zebo/tmp.signal_symbol.13d265489c33dfe98492b8cba043691d0e617a2c.cfg  
BRN_G0.ICE
CL_G0.CME
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/ef241158cf9e6124294afdbfa7864139809a270e/5cfb93ec2e3a460e355032ceb39b5b9a643d640c.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file:

In [8]:
for df in dfs:
    print display(df[:3])

,ap0,az0,ap1,az1,ap2,az2,ap3,az3,bp0,bz0,...,bp3,bz3,tp,ts,tv,time,sigA,sigB,sigC,sigD
time,,,,,,,,,,,,,,,,,,,,,
2017-11-01 02:02:00.041048,61.21,12.0,61.22,26.0,61.23,32.0,61.24,36.0,61.2,29.0,...,61.17,38.0,0.0,0.0,0.0,7.320041e+09,0.000019,0.000000e+00,-0.000006,0.000012
2017-11-01 02:02:00.041054,61.21,12.0,61.22,26.0,61.23,32.0,61.24,36.0,61.2,29.0,...,61.17,38.0,0.0,0.0,0.0,7.320041e+09,0.000019,0.000000e+00,-0.000006,0.000012
2017-11-01 02:02:00.041106,61.21,12.0,61.22,26.0,61.23,32.0,61.24,36.0,61.2,28.0,...,61.17,38.0,0.0,0.0,0.0,7.320041e+09,0.000017,3.580104e-13,-0.000006,0.000011


None


,ap0,az0,ap1,az1,ap2,az2,ap3,az3,bp0,bz0,...,bp3,bz3,tp,ts,tv,time,sigA,sigB,sigC,sigD
time,,,,,,,,,,,,,,,,,,,,,
2017-11-02 02:02:00.075478,60.65,38.0,60.66,22.0,60.67,26.0,60.68,19.0,60.64,23.0,...,60.61,35.0,0.0,0.0,0.0,7.320075e+09,-0.000012,0.000000e+00,0.000017,0.000006
2017-11-02 02:02:00.075722,60.65,38.0,60.66,22.0,60.67,26.0,60.68,19.0,60.64,23.0,...,60.61,35.0,0.0,0.0,0.0,7.320076e+09,-0.000012,-5.278362e-13,0.000017,0.000006
2017-11-02 02:02:00.142910,60.65,38.0,60.66,22.0,60.67,26.0,60.68,19.0,60.64,23.0,...,60.61,35.0,0.0,0.0,0.0,7.320143e+09,-0.000012,-1.363418e-13,0.000017,0.000006


None


,ap0,az0,ap1,az1,ap2,az2,ap3,az3,bp0,bz0,...,bp3,bz3,tp,ts,tv,time,sigA,sigB,sigC,sigD
time,,,,,,,,,,,,,,,,,,,,,
2017-11-03 02:02:00.073148,60.78,24.0,60.79,27.0,60.8,32.0,60.81,33.0,60.76,33.0,...,60.73,36.0,0.0,0.0,0.0,7.320073e+09,0.000008,1.012161e-08,-0.000017,-0.000009
2017-11-03 02:02:00.107164,60.78,24.0,60.79,27.0,60.8,32.0,60.81,33.0,60.76,33.0,...,60.73,36.0,0.0,0.0,0.0,7.320107e+09,0.000008,5.127202e-09,-0.000017,-0.000009
2017-11-03 02:02:00.107732,60.78,24.0,60.79,27.0,60.8,32.0,60.81,33.0,60.76,33.0,...,60.73,36.0,0.0,0.0,0.0,7.320108e+09,0.000008,5.071175e-09,-0.000017,-0.000009


...

None


,ap0,az0,ap1,az1,ap2,az2,ap3,az3,bp0,bz0,...,bp3,bz3,tp,ts,tv,time,sigA,sigB,sigC,sigD
time,,,,,,,,,,,,,,,,,,,,,
2017-11-09 02:02:00.090558,63.56,44.0,63.57,52.0,63.58,49.0,63.59,45.0,63.54,38.0,...,63.51,91.0,0.0,0.0,0.0,7.320091e+09,-0.000002,6.326289e-13,-0.000007,-0.000009
2017-11-09 02:02:00.226155,63.56,44.0,63.57,52.0,63.58,49.0,63.59,45.0,63.54,38.0,...,63.51,91.0,0.0,0.0,0.0,7.320226e+09,-0.000002,4.365327e-14,-0.000007,-0.000009
2017-11-09 02:02:00.400730,63.56,44.0,63.57,52.0,63.58,49.0,63.59,45.0,63.54,38.0,...,63.51,91.0,0.0,0.0,0.0,7.320401e+09,-0.000002,-5.309507e-13,-0.000007,-0.000009


None


,ap0,az0,ap1,az1,ap2,az2,ap3,az3,bp0,bz0,...,bp3,bz3,tp,ts,tv,time,sigA,sigB,sigC,sigD
time,,,,,,,,,,,,,,,,,,,,,
2017-11-10 02:02:00.003048,63.81,12.0,63.82,40.0,63.83,49.0,63.84,55.0,63.8,9.0,...,63.77,52.0,0.0,0.0,0.0,7.320003e+09,-0.000006,-0.000004,-0.000017,-0.000027
2017-11-10 02:02:00.015085,63.81,12.0,63.82,40.0,63.83,49.0,63.84,55.0,63.8,9.0,...,63.77,52.0,0.0,0.0,0.0,7.320015e+09,-0.000006,-0.000003,-0.000017,-0.000026
2017-11-10 02:02:00.017277,63.81,12.0,63.82,40.0,63.83,50.0,63.84,55.0,63.8,9.0,...,63.77,52.0,0.0,0.0,0.0,7.320017e+09,-0.000006,-0.000003,-0.000017,-0.000026


None


In [9]:
Horizon = 100
def fc_features(df):
    print "Generating Target..."
    mps = (df.ap0.values + df.bp0.values) * 0.5
    fmps = [np.array(mps[:Horizon]).sum()]
    for i in range(len(mps) - Horizon - 5):
        fmps.append(fmps[i] - mps[i] + mps[i+Horizon])
    diff = np.array(fmps) - np.array(mps[:len(fmps)])*Horizon
    df = df[:len(fmps)]
    df.insert(len(df.columns), 'target', pd.Series(diff, index=df.index))
    df = df[df.ap0 - df.bp0 <= 1.1*TICK_SIZE]
    
    print "Generating Book & Trade & signal features..."
    mat = df.values
    book_info = []
    trade_info = []
    sig_info = []
    for vec in mat:
        base_p = vec[0] - 4*TICK_SIZE
        tmp_vec = np.zeros(8)
        tmp_dic = {}
        for i in range(8):
            tmp_dic[int((vec[2*i] - base_p + 0.01 * TICK_SIZE)/TICK_SIZE)] = vec[2*i+1]
        for i in range(8):
            if i in tmp_dic.keys():
                tmp_vec[i] = tmp_dic[i]
        trade_level = int((vec[16] - base_p + 0.01 * TICK_SIZE)/TICK_SIZE)
        trade_vec = np.zeros(8)
        if trade_level >=4 and trade_level<8 and vec[17] > 0.5:
            trade_vec[trade_level] = vec[18]
        if trade_level >=0 and trade_level<4 and vec[17] < -0.5:
            trade_vec[trade_level] = vec[18]
        book_info.append(tmp_vec)
        trade_info.append(trade_vec)
        sig_info.append(np.array(vec[20:24])* 1.E7)
    print "Creating new training features..."
    reg_tar = df.target.values
    cls_tar = (reg_tar < -1E-9).astype(np.int16) + (reg_tar < +1E-9).astype(np.int16)
    DL_df = pd.DataFrame({
        'bk_info': book_info,
        'td_info': trade_info,
        'signals': sig_info,
        'reg_target': reg_tar,
        'cls_target': cls_tar
    })
    return DL_df

In [10]:
df_test = pd.DataFrame(dfs[0])
pool = MyPool(NUM_THREADS)
%time DL_dfs = pool.map(fc_features, dfs)
pool.close()
pool.join()
%time dl_df = fc_features(df_test)

Generating Target...
Generating Target...
Generating Book & Trade & signal features...
Generating Book & Trade & signal features...
Generating Target...
Generating Target...
Generating Book & Trade & signal features...
Generating Target...
Generating Book & Trade & signal features...
Generating Target...
Generating Book & Trade & signal features...
Generating Target...
Generating Target...
Generating Book & Trade & signal features...
Generating Book & Trade & signal features...
Generating Book & Trade & signal features...
Creating new training features...
Creating new training features...
Creating new training features...
Creating new training features...
Creating new training features...
Creating new training features...
Creating new training features...
Creating new training features...
CPU times: user 4min 13s, sys: 45.7 s, total: 4min 58s
Wall time: 16min 55s
Generating Target...
Generating Book & Trade & signal features...
Creating new training features...
CPU times: user 2min 1s,

In [11]:
for df in DL_dfs:
    print display(df[:3])

,bk_info,cls_target,reg_target,signals,td_info
0,"[38.0, 34.0, 25.0, 29.0, 12.0, 26.0, 32.0, 36.0]",0,0.210,"[187.071095547, 0.0, -63.2083917847, 123.86270...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[38.0, 34.0, 25.0, 29.0, 12.0, 26.0, 32.0, 36.0]",0,0.215,"[187.071095547, 0.0, -63.2083917847, 123.86270...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[38.0, 34.0, 25.0, 28.0, 12.0, 26.0, 32.0, 36.0]",0,0.220,"[174.530669221, 3.58010419891e-06, -63.2083917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


None


,bk_info,cls_target,reg_target,signals,td_info
0,"[35.0, 44.0, 28.0, 23.0, 38.0, 22.0, 26.0, 19.0]",1,9.094947e-13,"[-115.984519866, 0.0, 173.501338047, 57.516818...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[35.0, 44.0, 28.0, 23.0, 38.0, 22.0, 26.0, 19.0]",1,9.094947e-13,"[-115.984519866, -5.2783619668e-06, 173.501338...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[35.0, 44.0, 28.0, 23.0, 38.0, 22.0, 26.0, 19.0]",1,9.094947e-13,"[-115.984519866, -1.36341827168e-06, 173.50133...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


None


,bk_info,cls_target,reg_target,signals,td_info
0,"[33.0, 37.0, 34.0, 26.0, 1.0, 24.0, 27.0, 32.0]",1,-4.547474e-12,"[415.506199278, 352.555302811, -166.776103698,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[33.0, 37.0, 34.0, 26.0, 2.0, 24.0, 27.0, 32.0]",1,-4.547474e-12,"[415.506199278, 352.555302811, -166.776103698,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[33.0, 37.0, 34.0, 26.0, 2.0, 25.0, 27.0, 32.0]",1,-4.547474e-12,"[415.506199278, 352.555302811, -166.776103698,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


...

None


,bk_info,cls_target,reg_target,signals,td_info
0,"[92.0, 71.0, 57.0, 36.0, 1.0, 43.0, 52.0, 49.0]",1,-9.094947e-12,"[422.581827871, 352.546677233, -112.490494171,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[92.0, 71.0, 57.0, 36.0, 1.0, 43.0, 52.0, 49.0]",1,-9.094947e-12,"[422.581827871, 349.821595675, -112.490494171,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[92.0, 71.0, 57.0, 31.0, 1.0, 43.0, 52.0, 49.0]",1,-9.094947e-12,"[418.87477617, 349.821595675, -112.490494171, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


None


,bk_info,cls_target,reg_target,signals,td_info
0,"[52.0, 44.0, 43.0, 9.0, 12.0, 40.0, 49.0, 55.0]",1,2.728484e-12,"[-58.2490970898, -41.5463814861, -173.45683746...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[52.0, 44.0, 43.0, 9.0, 12.0, 40.0, 49.0, 55.0]",1,2.728484e-12,"[-58.2490970898, -32.7648090114, -173.45683746...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[52.0, 44.0, 43.0, 9.0, 12.0, 40.0, 50.0, 55.0]",1,2.728484e-12,"[-58.2490970898, -31.3886349478, -173.45683746...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


None
